In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, MaxPool2D, GlobalAvgPool2D

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array

In [2]:
# Resizing all image

IMAGE_SIZE = [224, 224]

batch_size = 5

In [3]:
train_path = 'D:\Object Detection\Mask-Nomask detection-real time\Datasets\Train'

In [4]:
os.listdir(train_path)

['Mask', 'No mask']

### Creating Image Data generators and Training Set

In [5]:
train_image_gen = ImageDataGenerator(rotation_range=20, shear_range=0.2, zoom_range=0.2,
                                     horizontal_flip= True, rescale= 1/255.0)

In [6]:
training_set = train_image_gen.flow_from_directory(train_path, target_size=(224, 224), class_mode='binary', batch_size= 5)

Found 30 images belonging to 2 classes.


### Creating the model

In [7]:
mobnetv2 = MobileNetV2(input_shape= [224, 224, 3] , alpha=1.0, include_top= False,
                       weights='imagenet', classifier_activation='softmax')

x = mobnetv2.output
x = GlobalAvgPool2D()(x)
x = Dense(30, activation='relu')(x)
x = Flatten()(x)

predictions = Dense(training_set.num_classes, activation= 'softmax')(x)

model = Model(inputs = mobnetv2.inputs, outputs = predictions)

for layer in mobnetv2.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [9]:
model.fit(training_set, epochs= 15, batch_size = 5)

Epoch 1/15
6/6 [==============================] - 7s 272ms/step - loss: 0.5229 - accuracy: 0.7486
Epoch 2/15
6/6 [==============================] - 2s 266ms/step - loss: 0.1130 - accuracy: 0.9776
Epoch 3/15
6/6 [==============================] - 2s 268ms/step - loss: 0.0355 - accuracy: 1.0000
Epoch 4/15
6/6 [==============================] - 2s 271ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 5/15
6/6 [==============================] - 2s 268ms/step - loss: 0.0169 - accuracy: 1.0000
Epoch 6/15
6/6 [==============================] - 2s 272ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 7/15
6/6 [==============================] - 2s 270ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 8/15
6/6 [==============================] - 2s 262ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 9/15
6/6 [==============================] - 2s 272ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 10/15
6/6 [==============================] - 2s 270ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 11/15
6/6 [==

In [10]:
model.save('mask_detection.h5')

In [11]:
new_model = tf.keras.models.load_model('mask_detection.h5')

## Testing the model with a live video

In [12]:
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

In [ ]:
font =  cv2.FONT_HERSHEY_COMPLEX

video_capture = cv2.VideoCapture(0)

def face_extractor(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(image, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = image[y:y+h, x:x+w]
        return cropped_face

while True:
    ret, frame = video_capture.read()
    
    face= face_extractor(frame)
    if type(face) == np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
                    #Our keras model used a 4D tensor, (images x height x width x channel)
                    #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = new_model.predict(img_array)
        print(pred)
        
        if pred[0][0]> 0.5:
            status = 'nomasks'
            cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2, cv2.LINE_4)
        
    else:
        status = 'mask'
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255),2, cv2.LINE_4)
        
    cv2.imshow('Video', frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

[[0.264098 0.735902]]
[[0.35934505 0.640655  ]]
[[0.26926506 0.7307349 ]]
[[0.5096613  0.49033877]]
[[0.27395064 0.72604936]]
[[0.27632716 0.72367287]]
[[0.29488605 0.70511395]]
[[0.45386112 0.5461389 ]]
[[0.44080442 0.55919564]]
[[0.25072673 0.74927336]]
[[0.21002609 0.7899739 ]]
[[0.39823848 0.60176146]]
[[0.3031465  0.69685346]]
[[0.35102606 0.6489739 ]]
[[0.3022361  0.69776386]]
[[0.4254649  0.57453513]]
[[0.36753145 0.6324686 ]]
[[0.40483588 0.5951642 ]]
[[0.5433745 0.4566255]]
[[0.58387846 0.41612154]]
[[0.6104735  0.38952646]]
[[0.47752652 0.52247345]]
[[0.694262   0.30573794]]
[[0.38142926 0.6185707 ]]
[[0.6501927 0.3498073]]
[[0.5601247 0.4398753]]
[[0.59267807 0.40732187]]
[[0.70232517 0.29767483]]
[[0.5473827  0.45261732]]
[[0.50081015 0.4991898 ]]
[[0.61120325 0.3887968 ]]
[[0.6668504 0.3331496]]
[[0.55531156 0.44468844]]
[[0.53563976 0.46436024]]
[[0.48641893 0.5135811 ]]
[[0.7003469  0.29965305]]
[[0.573286 0.426714]]
[[0.52381706 0.4761829 ]]
[[0.67078   0.3292201]]
[[0.

[[0.71517473 0.2848252 ]]
[[0.6113926  0.38860738]]
[[0.40697578 0.5930242 ]]
[[0.49249038 0.5075096 ]]
[[0.54573894 0.45426103]]
[[0.5173126  0.48268747]]
[[0.50183976 0.49816027]]
[[0.60650873 0.3934912 ]]
[[0.32426775 0.6757322 ]]
[[0.4615435 0.5384565]]
[[0.6202116  0.37978843]]
[[0.5299284  0.47007164]]
[[0.5809174  0.41908255]]
[[0.42403883 0.5759612 ]]
[[0.5587027  0.44129726]]
[[0.38329497 0.616705  ]]
[[0.61555934 0.3844407 ]]
[[0.40538025 0.59461975]]
[[0.4974057  0.50259435]]
[[0.28040573 0.71959424]]
[[0.4874543  0.51254565]]
[[0.4848071 0.5151929]]
[[0.51595277 0.48404723]]
[[0.54664314 0.45335686]]
[[0.45659885 0.5434011 ]]
[[0.72311866 0.27688137]]
[[0.54351234 0.45648763]]
[[0.48881137 0.5111887 ]]
[[0.28171298 0.718287  ]]
[[0.41977414 0.5802259 ]]
[[0.33888143 0.66111857]]
[[0.431445 0.568555]]
[[0.44217715 0.55782294]]
[[0.59041    0.40959004]]
[[0.60452914 0.39547083]]
[[0.79518586 0.20481418]]
[[0.7138059 0.2861941]]
[[0.38453183 0.61546826]]
[[0.81575996 0.1842400